Firstly [Downloads models](https://github.com/spmallick/learnopencv/tree/master/FaceDetectionComparison/models)

In [1]:
# Setup detectron2 logger
from detectron2.utils.logger import setup_logger
setup_logger()

# import some common libraries
import numpy as np

from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg

In [11]:
class Detector:
    def __init__(self, plugin_config = None):
        if plugin_config is None:
            plugin_config = {}
        cfg = get_cfg()
        # add project-specific config (e.g., TensorMask) here if you're not running a model in detectron2's core library
        cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml"))
        cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5  # set threshold for this model
        # Find a model from detectron2's model zoo. You can use the https://dl.fbaipublicfiles... url as well
        cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml")
        self.predictor = DefaultPredictor(cfg)

    def check(self, img):
       outputs = self.predictor(img)
       contours = []
       clases   = []
       bboxs    = []
       scores   = []
       for mask, cls, bbox, score in zip(outputs["instances"].pred_masks, \
                                  outputs["instances"].pred_classes, \
                                  outputs["instances"].pred_boxes, \
                                  outputs["instances"].scores):
           cls   = int(cls.to("cpu"))
           score = float(score.to("cpu"))
           bbox  = [int(x) for x in bbox.to("cpu")]
           poly, hierarchy = cv2.findContours(
               np.array(mask.to("cpu")).astype(np.uint8),
               cv2.RETR_TREE,
               cv2.CHAIN_APPROX_SIMPLE
               )
           poly = np.array([cnt for cnt, h in zip(poly, hierarchy[0]) \
                           if ((len(np.unique(cnt, axis=0)) > 3) and (h[3] == -1))], dtype=object)
           clases.append(cls)
           contours.append(poly)
           bboxs.append(bbox)
           scores.append(score)
       return clases, bboxs, contours, scores

In [12]:
import cv2
from matplotlib import pyplot as plt
plt.rcParams["figure.figsize"] = (20,20)

In [13]:
print("[INFO] loading model...")
net = cv2.dnn.readNetFromCaffe("deploy.prototxt", "res10_300x300_ssd_iter_140000_fp16.caffemodel")

[INFO] loading model...


In [14]:
detector = Detector()

In [15]:
def detect_face(image, min_confidence=0.7):
    (h, w) = image.shape[:2]
    blob = cv2.dnn.blobFromImage(cv2.resize(person_img, (300, 300)), 1.0, (300, 300), (104.0, 177.0, 123.0))
    net.setInput(blob)
    detections = net.forward()
    detected_faces = []
    # loop over the detections
    for i in range(0, detections.shape[2]):
        # extract the confidence (i.e., probability) associated with the
        # prediction
        confidence = detections[0, 0, i, 2]
        # filter out weak detections by ensuring the `confidence` is
        # greater than the minimum confidence
        if confidence > min_confidence:
            # compute the (x, y)-coordinates of the bounding box for the
            # object
            box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
            (startX, startY, endX, endY) = box.astype("int")
            detected_faces.append([startX, startY, endX, endY])
    return image, detected_faces

In [16]:
def to_rect_bbox(bboxs_item, w, h):
    bw = bboxs_item[2]-bboxs_item[0]
    bh = bboxs_item[3]-bboxs_item[1]
    
    y1 = bboxs_item[1]
    y2 = bboxs_item[3]
    x1 = bboxs_item[0]
    x2 = bboxs_item[2]
        
    if bw > bh:
        diff = (bw-bh)/2
        y1 -= diff
        y2 += diff
        if y1 < 0:
            y2 -=  y1
            y1 = 0
        if y2 > h:
            y1 -= h-y2
            y2 = h
        if y1 < 0:
            y1 = 0
    if bw < bh:
        diff = (bh-bw)/2
        x1 -= diff
        x2 += diff
        if x1 < 0:
            x2 -=  x1
            x1 = 0
        if x2 > w:
            x1 -= w-x2
            x2 = w
        if x1 < 0:
            x1 = 0
    return int(x1), int(x2), int(y1), int(y2)

In [17]:
def anonymize_face_pixelate(image, blocks=5):
    # divide the input image into NxN blocks
    (h, w) = image.shape[:2]
    xSteps = np.linspace(0, w, blocks + 1, dtype="int")
    ySteps = np.linspace(0, h, blocks + 1, dtype="int")
    # loop over the blocks in both the x and y direction
    for i in range(1, len(ySteps)):
        for j in range(1, len(xSteps)):
            # compute the starting and ending (x, y)-coordinates
            # for the current block
            startX = xSteps[j - 1]
            startY = ySteps[i - 1]
            endX = xSteps[j]
            endY = ySteps[i]
            # extract the ROI using NumPy array slicing, compute the
            # mean of the ROI, and then draw a rectangle with the
            # mean RGB values over the ROI in the original image
            roi = image[startY:endY, startX:endX]
            (B, G, R) = [int(x) for x in cv2.mean(roi)[:3]]
            cv2.rectangle(image, (startX, startY), (endX, endY),
                (B, G, R), -1)
    # return the pixelated blurred image
    return image

In [ ]:
import glob
import tqdm

debug = 1
all_photos = 0
with_people = 0
g = glob.glob("/var/www/nomeroff-net/yolov5/data_source/many_line/*")
for img_path in tqdm.tqdm(g):
    try:
        img = cv2.imread(img_path)
        if img is None:
            raise Exception("None")
    except:
        continue
    h, w, c = img.shape
    clases, bboxs, contours, scores = detector.check(img)
    pretty_clases = []
    for clases_item, bboxs_item, contours_item, scores_item in zip(clases, bboxs, contours, scores):
        bw = bboxs_item[2]-bboxs_item[0]
        bh = bboxs_item[3]-bboxs_item[1]
        x1, x2, y1, y2 = to_rect_bbox(bboxs_item, w, h)
        area = (bw*bh)/(w*h)
        if scores_item > 0.7 and clases_item==0:
            person_img = img[y1:y2, x1:x2]
            person_img, detected_faces = detect_face(person_img)
            for face_bbox in detected_faces:
                face_x1, face_y1, face_x2, face_y2 = face_bbox
                face_x1 += x1
                face_y1 += y1
                face_x2 += x1
                face_y2 += y1
                img[face_y1:face_y2, face_x1:face_x2] = anonymize_face_pixelate(
                    img[face_y1:face_y2, face_x1:face_x2]
                )
            if len(detected_faces):
                pretty_clases.append(clases_item)
    if 0 in pretty_clases:
        with_people += 1
        if debug:
            plt.imshow(img[:,:,::-1])
            plt.show()
        else:
            img = cv2.imwrite(img_path, img)
    all_photos += 1
    #break
print("all_photos", all_photos)
print("with_people", with_people)